## Table of contents

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ol>
        <li><a href="#1.-Importing-Required-Libraries-and-Data"> Importing Required Libraries and Data</a></li>
        <li><a href="#2.-Data-Pre-processing-and-Preparation"> Data Pre-processing and Preparation</a></li>
        <li><a href="#3.-verison_1 - Search Engine Using Fuzzy-Wuzzy Library"> Search Engine Using Fuzzy-Wuzzy Library</a></li>
        </li>
    </ol>
</div>

## Importing Libraries and Data

In [141]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

import rapidfuzz
from rapidfuzz import process, utils

import spacy
from tqdm import tqdm

from re import sub
from gensim.utils import simple_preprocess

from nltk.tokenize import word_tokenize
import nltk

from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

import spacy
from rank_bm25 import BM25Okapi
from tqdm import tqdm
import timeit

import pandas as pd

In [10]:
#importing the file

dashboards = pd.read_csv('microstrategy_and_dashboard.csv')
metrics = pd.read_csv('tableau_metric.csv')

In [11]:
dashboards.head()

,id,dashboards,caption,datasource,url,Global_usage,users_id_1,user_id_2
0,139444,Ad Analysis,Beer Style Level Value|Beer Style Levels|Beer ...,IRI 52wk 34.0,https:andanddashboards.armadilloinsight.comand...,1717,532,1185
1,139443,Ad Recap,Beer Style Level Value|Beer Style Levels|Beer ...,IRI 52wk 34.0,https:andanddashboards.armadilloinsight.comand...,974,523,451
2,139434,Aggregate Sales per Pt vs. CWD,Beer Style Level Value|Beer Style Levels|Beer ...,IRI 52wk 34.0|IRI Agg 34.0,https:andanddashboards.armadilloinsight.comand...,851,422,429
3,139466,Brand Control,Brand Family|Brand Levels|Brand|Main Brand Fam...,IRI 52wk 34.0,https:andanddashboards.armadilloinsight.comand...,923,625,298
4,139465,Brand In and Out Scorecard,Segment|Time as Date,IRI 52wk 34.0,https:andanddashboards.armadilloinsight.comand...,876,520,356


In [12]:
metrics.head()

,id,metric_name,descriptors
0,0,Sales Measure Selection,Dollar Sales|Dollar Sales|Dollar Sales Change ...
1,1,Brand Levels,Brand Family|Vendor|Brand Family|Brand|Sku|Seg...
2,3,Beer Style Levels,Beer Style Value|Macro Style Value|Mezzo Style...
3,4,Brands and Skew Value,Brand Family|Brand|Segment
4,5,Beer Style,AMBER ALE|AMBER LAGE|BARLEY WINE|BELGIAN ALES|...


In [13]:
#converting column values to lowercase

dashboards = dashboards.apply(lambda x: x.astype(str).str.lower())
metrics = metrics.apply(lambda x: x.astype(str).str.lower())

In [14]:
#splitting caption and descriptors columns 

dashboards = dashboards[['dashboards','caption','Global_usage']]
dashboards['metric_names'] = dashboards.caption.str.split('|')
metrics['descriptors'] = metrics.descriptors.str.split('|')

In [15]:
#converting metric names and descriptors in each row values to multiple rows

spl_dashboards = dashboards.set_index(['dashboards'])['metric_names'].apply(pd.Series).stack().reset_index().drop('level_1', axis=1).rename(columns={0:'caption'})
spl_metrics = metrics.set_index(['metric_name'])['descriptors'].apply(pd.Series).stack().reset_index().drop('level_1', axis=1).rename(columns={0:'descriptors'})


In [16]:
#merging dashboards, metrics and descriptors

master = spl_dashboards.merge(spl_metrics, how = 'inner', left_on ='caption', right_on = 'metric_name')[['dashboards','caption','descriptors']]
master = master.rename(columns={"caption": "metrics"})
master


,dashboards,metrics,descriptors
0,ad analysis,beer style level value,macro style value
1,ad analysis,beer style level value,mezzo style value
2,ad analysis,beer style level value,micro style valu
3,ad analysis,beer style level value,beer style value
4,ad analysis,beer style level value,altbier
...,...,...,...
236553,surveyed stores,surveyed stores,2018
236554,surveyed stores,surveyed stores,2019
236555,surveyed stores,surveyed stores,2020
236556,surveyed stores,surveyed stores,2021


# Creating Corpus

In [17]:
#creating corpus

dashboards_list = list(master.dashboards.unique())
metrics_list = list(master.metrics.unique())
descriptors_list = list(master.descriptors.unique())
mastercorpus = dashboards_list + metrics_list + descriptors_list

In [104]:
#pre-processing corpus

mastercorpus = list(filter(None,mastercorpus))

### Pre-Processing Steps :
- stop words
- lower case
- stemming
- lemmatization
- noun filter 

### Search Suggestions flow :
- Fuzzywuzzy suggestion (includes dashboard title, metric names and descriptors)
- Dashboard names extraction 
- Final suggestions (popularity of dashboard)

### Efficient approaches:

- storing metric and descriptors as keys, and dashboards names as values in a dictionary 
- weightage to dashboard names, followed by metric and descriptors

# Basline Model 
## Method 1: RegEx string matching

In [101]:
import re
from re import match

In [194]:
start = timeit.default_timer()
r = re.compile(".*mezzo")
newlist = list(filter(r.match, mastercorpus))
stop = timeit.default_timer()
execution_time = stop - start

print("Program Executed in "+str(execution_time))
print(newlist)

Program Executed in 0.006374348000463215
['mezzo style value', 'mezzo spritz virtue cider brand family', 'café intermezzo nashville (97587)']


### Disadvantage: 
It won't be able to handle the spelling mistakes from the user. 

## Method 2: Using fuzzywuzzy Library

In [195]:
def fuzzywuzzy_suggestion(corpus, input_query):
    for i in [corpus]:
        return process.extract(input_query, i, limit=10)
        
def dashboard_names_suggestion(dashboards, Fuzzy_suggestions):
    start_time = time.time()
    dashboard_names = []
       
    for i in Fuzzy_suggestions:
        if i[0] in dashboards_list:  
            dashboard_names.append(i[0])
        elif i[0] in metrics_list:   
            dashboard_names = dashboard_names + (dashboards.loc[(dashboards.metrics.str.lower() == i[0].lower())].dashboards).to_list()
        else: 
            dashboard_names = dashboard_names + (dashboards.loc[(dashboards.descriptors.str.lower() == i[0].lower())].dashboards).to_list()
            end_time = time.time()
    return dashboard_names


In [196]:
# Input query:
start = timeit.default_timer()
suggestion = fuzzywuzzy_suggestion(mastercorpus,'sales')
print(suggestion)

dashboard_suggestions = list(dashboard_names_suggestion(master,suggestion))

search_output = []
search_output = list(dict.fromkeys(dashboard_suggestions))


stop = timeit.default_timer()
execution_time = stop - start

print("Program Executed in "+str(execution_time))
search_output[:10]

[('aggregate sales per pt vs. cwd', 90.0, 2), ('sales per pt vs. cwd', 90.0, 32), ('sales team opportunitshis year ', 90.0, 55), ('sales manager dashboard', 90.0, 56), ('sales team tracker ', 90.0, 59), ('sales team gains and losses', 90.0, 60), ('sales measure selection', 90.0, 84), ('sales state', 90.0, 130), ('dollar sales', 90.0, 802), ('dollar sales change ya', 90.0, 803)]
Program Executed in 0.4486926549998316


['aggregate sales per pt vs. cwd',
 'sales per pt vs. cwd',
 'sales team opportunitshis year ',
 'sales manager dashboard',
 'sales team tracker ',
 'sales team gains and losses',
 'ad analysis',
 'r geography over time',
 'category and segment analysis',
 'competitive set']

## Method 3:  Using FuzzyWuzzy with Global Usage of Dashboards 

In [24]:
dashboards.Global_usage = pd.to_numeric(dashboards.Global_usage)

In [25]:
dashboards[dashboards['dashboards'].isin(search_output)].sort_values(by = 'Global_usage',ascending = False)
dashboards.head()

,dashboards,caption,Global_usage,metric_names
0,ad analysis,beer style level value|beer style levels|beer ...,1717,"[beer style level value, beer style levels, be..."
1,ad recap,beer style level value|beer style levels|beer ...,974,"[beer style level value, beer style levels, be..."
2,aggregate sales per pt vs. cwd,beer style level value|beer style levels|beer ...,851,"[beer style level value, beer style levels, be..."
3,brand control,brand family|brand levels|brand|main brand fam...,923,"[brand family, brand levels, brand, main brand..."
4,brand in and out scorecard,segment|time as date,876,"[segment, time as date]"


In [153]:
def popular_dashboard_suggestion(dashboards, dashboardSuggestion): 
    return dashboards[dashboards['dashboards'].isin(dashboardSuggestion)].sort_values(by = 'Global_usage',
                                                                      ascending = False)[['dashboards',
                                                                                          'Global_usage']]            

In [154]:
#popular dashboard suggestions
start = timeit.default_timer()
x = popular_dashboard_suggestion(dashboards,search_output).head(10)
stop = timeit.default_timer()
execution_time = stop - start

print("Program Executed in "+str(execution_time))
print(x)


Program Executed in 0.00295383499997115
                       dashboards  Global_usage
16         dollar ranker with cwd          7540
12  category and segment analysis          4583
15               dollar rank comp          4255
42                 style analysis          3556
40                        rankers          2535
36                 price analysis          2114
24          market index analysis          2012
0                     ad analysis          1717
6             brewery comparisons          1712
32                     our brands          1456


In [26]:
#df_new[df_new.caption.str.lower() == 'select main vendor']

In [27]:
# checking dashboard_names for metric = 'segment'
#string = 'Geography'.lower()
#print(df_new.loc[df_new.caption.str.lower() == string])
#print(df_new.loc[df_new.caption.str.lower() == 'geography'].count())

In [28]:
#dashboards[dashboards['dashboards'].isin(dasboard_names_2)].sort_values(by = 'popularity', ascending = False)

In [29]:
#popular_suggestions = popular_dashboard_suggestion(df,dasboard_names_2)
#print(len(popular_suggestions))
#popular_suggestions

In [30]:
# dashboard_names=[]
# for i in suggestion:
#         print(i[0])
#         for key, value in dict_dashboard_metric.items():
#                 for v in value:
# #                     print(v)
# #                     print(type(i[0]),type(v))
#                     if i[0] == v:
# #                         print('no')
#                         dashboard_names.append(key)
# dashboard_names

# def dashboard_name_suggestion(suggestion, dict1,dict2):
#     list_of_keys =[]
#     for value in suggestion:
      
#         list_of_keys.append([key for key, list_of_values in dict1.items()
#                 if value[0] in list_of_values])
#         list_of_keys.append([key for key, list_of_values in dict2.items()
#                 if value[0] in list_of_values])
#         return list_of_keys

## Method 4: Using auto-suggestion of dashboard names

In [124]:
from fast_autocomplete import AutoComplete

In [128]:
dashboard_dict = dict(enumerate(master.dashboards.unique().flatten(), 1))
dashboard_dict = {v: k  for k,v in dashboard_dict.items()}
dashboard_dict

metric_dict = dict(enumerate(master.metrics.unique().flatten(), 1))
metric_dict = {v: k  for k,v in metric_dict.items()}
metric_dict

descriptor_dict = dict(enumerate(master.descriptors.unique().flatten(), 1))
descriptor_dict = {v: k  for k,v in descriptor_dict.items()}
descriptor_dict


# metric_dict = df_final.caption.unique().to_dict()
# descriptor_dict = df_final.descriptor.unique().to_dict()

{'macro style value': 1,
 'mezzo style value': 2,
 'micro style valu': 3,
 'beer style value': 4,
 'altbier': 5,
 'american amber or red ale': 6,
 'american brown ale': 7,
 'american ipa': 8,
 'american lager': 9,
 'american light lager': 10,
 'american pale le': 11,
 'american porter': 12,
 'american stout': 13,
 'american wild ale': 14,
 'anniversary release': 15,
 'baltic porter': 16,
 'barleywine': 17,
 'barrel aged specialty': 18,
 'beer flagship plus mix pack': 19,
 'belgian upa': 20,
 'belgian pale ale': 21,
 'berliner weisse': 22,
 'blonde ale': 23,
 'bock': 24,
 'california common beer': 25,
 'citrus flavored neutral base': 26,
 'collaboration beer': 27,
 'cream ale': 28,
 'doppelbock': 29,
 'dubbel': 30,
 'dunkel': 31,
 'english brown ale': 32,
 'english foreign or export stout': 33,
 'english ia': 34,
 'english pale ale': 35,
 'esb': 36,
 'flanders': 37,
 'fruit ipa': 38,
 'fruit specialty': 39,
 'gose or gueuze': 40,
 'hefeweizen': 41,
 'helles': 42,
 'herb or spice special

In [129]:
masterdict = {}
masterdict.update(dashboard_dict)
masterdict.update(metric_dict)
masterdict.update(descriptor_dict)

In [131]:
for k,v in masterdict.items():
    masterdict[k] = {}

In [182]:
autocomplete = AutoComplete(words = masterdict)

In [188]:
autosuggestions = autocomplete.search(word = 'sales', max_cost= 3, size = 100)
start = timeit.default_timer()
dashboard_suggestions_auto = dashboard_names_suggestion(master,autosuggestions)

search_output = []
search_output = list(dict.fromkeys(dashboard_suggestions_auto))

stop = timeit.default_timer()
execution_time = stop - start

print("Program Executed in "+str(execution_time))
search_output[:10]

Program Executed in 0.24595099600082904


['depletions month cube with inventory and shipmentments',
 'depletionsletions cube',
 'retail month cube',
 'retail cube to  brand quality level',
 'retail cube to brand level',
 'retail cube with reorder to rollup',
 'sales team opportunitshis year ',
 'sales manager dashboard',
 'depletionsletions',
 'sales team tracker ']

## Method 5:  Wordnet

## Comparisions

# Using TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

search_terms = 'dollar sales'
documents = text

doc_vectors = TfidfVectorizer().fit_transform([search_terms] + documents)

cosine_similarities = linear_kernel(doc_vectors[0:1], doc_vectors).flatten()
document_scores = [item.item() for item in cosine_similarities[1:]]

tuples = list(zip(text,document_scores))
df_scores = pd.DataFrame(tuples, columns = ['text','document_scores'])
df_scores.sort_values('document_scores', ascending =False).head(10)

In [ ]:
doc_vectors.nonzero()

# Using Glove's Model

In [ ]:
from re import sub
from gensim.utils import simple_preprocess

query_string = 'area'
documents = text

stopwords = ['the', 'and', 'are', 'a']

# From: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/soft_cosine_tutorial.ipynb
def preprocess(doc):
    # Tokenize, clean up input document string
    doc = sub(r'<img[^<>]+(>|$)', " image_token ", doc)
    doc = sub(r'<[^<>]+(>|$)', " ", doc)
    doc = sub(r'\[img_assist[^]]*?\]', " ", doc)
    doc = sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', " url_token ", doc)
    return [token for token in simple_preprocess(doc, min_len=0, max_len=float("inf")) if token not in stopwords]

# Preprocess the documents, including the query string
corpus = [preprocess(document) for document in documents]
query = preprocess(query_string)




# Build the term dictionary, TF-idf model
dictionary = Dictionary(corpus+[query])
tfidf = TfidfModel(dictionary=dictionary)

# Create the term similarity matrix.  
similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary, tfidf)

# Compute Soft Cosine Measure between the query and the documents.
# From: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/soft_cosine_tutorial.ipynb
query_tf = tfidf[dictionary.doc2bow(query)]

index = SoftCosineSimilarity(
            tfidf[[dictionary.doc2bow(document) for document in corpus]],
            similarity_matrix)

doc_similarity_scores = index[query_tf]

# Output the sorted similarity scores and documents
sorted_indexes = np.argsort(doc_similarity_scores)[::-1]
for idx in sorted_indexes:
    print(f'{idx} \t {doc_similarity_scores[idx]:0.3f} \t {documents[idx]}')


In [ ]:
# import gensim.downloader as api
# from gensim.corpora import Dictionary
# from gensim.models import TfidfModel
# from gensim.models import WordEmbeddingSimilarityIndex
# from gensim.similarities import SparseTermSimilarityMatrix
# from gensim.similarities import SoftCosineSimilarity

# # Load the model: this is a big file, can take a while to download and open
# glove = api.load("glove-wiki-gigaword-50")    
# similarity_index = WordEmbeddingSimilarityIndex(glove)



In [ ]:
# Build the term dictionary, TF-idf model
dictionary = Dictionary(corpus+[query])
tfidf = TfidfModel(dictionary=dictionary)

# Create the term similarity matrix.  
similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary, tfidf)

# Compute Soft Cosine Measure between the query and the documents.
# From: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/soft_cosine_tutorial.ipynb
query_tf = tfidf[dictionary.doc2bow(query)]

index = SoftCosineSimilarity(
            tfidf[[dictionary.doc2bow(document) for document in corpus]],
            similarity_matrix)

doc_similarity_scores = index[query_tf]

# Output the sorted similarity scores and documents
sorted_indexes = np.argsort(doc_similarity_scores)[::-1]
for idx in sorted_indexes:
    print(f'{idx} \t {doc_similarity_scores[idx]:0.3f} \t {documents[idx]}')
